In [1]:
import os
import cv2
import numpy as np
from skimage.io import imread
from skimage.transform import resize
from skimage.feature import hog
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier

In [2]:
import os
from zipfile import ZipFile


!pip install kaggle


os.environ['KAGGLE_USERNAME'] = "majisouvik1099"
os.environ['KAGGLE_KEY'] = "9464225218d56f8bfea9dd9cc437489f"


!kaggle datasets download -d jessicali9530/lfw-dataset


with ZipFile('lfw-dataset.zip', 'r') as zip_ref:
    zip_ref.extractall('lfw-dataset')


os.remove('lfw-dataset.zip')


 82% 92.0M/112M [00:00<00:00, 192MB/s]
100% 112M/112M [00:00<00:00, 178MB/s] 


In [3]:

def extract_features(image_path, model):
    image = Image.open(image_path).convert('RGB')
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    image = preprocess(image)
    image = image.unsqueeze(0)
    with torch.no_grad():
        features = model(image)
    features = features.squeeze(0)
    return features


def compute_hog(img):
    resized_img = resize(img, (128*4, 64*4))
    fd, hog_image = hog(resized_img, orientations=9, pixels_per_cell=(8, 8),
                    cells_per_block=(2, 2), visualize=True, multichannel=True)
    return fd


def get_pixel(img, center, x, y):
    new_value = 0
    try:
        if img[x][y] >= center:
            new_value = 1
    except:
        pass
    return new_value

def lbp_calculated_pixel(img, x, y):
    center = img[x][y]
    val_ar = []
    val_ar.append(get_pixel(img, center, x-1, y+1))
    val_ar.append(get_pixel(img, center, x, y+1))
    val_ar.append(get_pixel(img, center, x+1, y+1))
    val_ar.append(get_pixel(img, center, x+1, y))
    val_ar.append(get_pixel(img, center, x+1, y-1))
    val_ar.append(get_pixel(img, center, x, y-1))
    val_ar.append(get_pixel(img, center, x-1, y-1))
    val_ar.append(get_pixel(img, center, x-1, y))

    power_val = [1, 2, 4, 8, 16, 32, 64, 128]
    val = 0
    for i in range(len(val_ar)):
        val += val_ar[i] * power_val[i]
    return val

def calcLBP(img):
    height, width, channel = img.shape
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_lbp = np.zeros((height, width,3), np.uint8)
    for i in range(0, height):
        for j in range(0, width):
             img_lbp[i, j] = lbp_calculated_pixel(img_gray, i, j)
    hist_lbp = cv2.calcHist([img_lbp], [0], None, [256], [0, 256])
    return hist_lbp.flatten()


In [4]:
resnet = models.resnet50(pretrained=True)
resnet = nn.Sequential(*list(resnet.children())[:-1])
resnet.eval()

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 84.5MB/s]


Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)


In [5]:
lfw_folder = '/content/lfw-dataset/lfw-deepfunneled/lfw-deepfunneled'
X, y = [], []
for folder_name in os.listdir(lfw_folder):
    folder_path = os.path.join(lfw_folder, folder_name)
    if os.path.isdir(folder_path):
        num_images = len(os.listdir(folder_path))
        if num_images > 70:
            for image_name in os.listdir(folder_path):
                image_path = os.path.join(folder_path, image_name)
                image = imread(image_path)
                hog_feature = compute_hog(image)
                # lbp_feature = calcLBP(image)
                # cnn_feature = extract_features(image_path, resnet).numpy()
                hog_feature = hog_feature.reshape(-1)
                # lbp_feature = lbp_feature.reshape(-1)
                # cnn_feature = cnn_feature.flatten()
                # combined_feature = np.concatenate((hog_feature, lbp_feature, cnn_feature))

                X.append(hog_feature)
                y.append(folder_name)


<ipython-input-3-8d0f7df1f55f>:19: FutureWarning: `multichannel` is a deprecated argument name for `hog`. It will be removed in version 1.0. Please use `channel_axis` instead.
  fd, hog_image = hog(resized_img, orientations=9, pixels_per_cell=(8, 8),


In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=30)

In [7]:
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

In [8]:
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV
parameters = [
        {"kernel":
            ["linear"],
            "C": [ 0.1, 1, 10, 100]},
        {"kernel":
            ["poly"],
            "degree": [2, 3, 4],
            "C": [ 0.1, 1, 10, 100]},
        {"kernel":
            ["rbf"],
            "gamma": ["auto", "scale"],
            "C": [ 0.1, 1, 10, 100]}
]

svm_clf = SVC()
print("tuning hyperparameters via grid search")
grid = GridSearchCV(estimator=SVC(), param_grid=parameters, n_jobs=-1)
grid.fit(X_train,y_train_encoded)
print(f"grid search best score: {grid.best_score_ * 100:.2f}%")
print(f"grid search best parameters: {grid.best_params_}")



tuning hyperparameters via grid search


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()
/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()


grid search best score: 88.74%
grid search best parameters: {'C': 0.1, 'kernel': 'linear'}


In [9]:
model = grid.best_estimator_
y_pred_svm = model.predict(X_test)
# print(classification_report(y_test_encoded, y_pred_svm))
print("SVM Classifier")
print("Classification Report:")
print(classification_report(y_test_encoded, y_pred_svm, target_names=label_encoder.classes_))

SVM Classifier
Classification Report:
                   precision    recall  f1-score   support

     Ariel_Sharon       0.92      0.79      0.85        14
     Colin_Powell       0.93      0.95      0.94        40
  Donald_Rumsfeld       1.00      0.79      0.89        34
    George_W_Bush       0.84      1.00      0.91       109
Gerhard_Schroeder       1.00      0.78      0.88        23
      Hugo_Chavez       1.00      0.64      0.78        11
       Tony_Blair       1.00      0.85      0.92        27

         accuracy                           0.90       258
        macro avg       0.95      0.83      0.88       258
     weighted avg       0.92      0.90      0.90       258



In [10]:
y_pred_svm = model.predict(X_train)
accuracy_svm = accuracy_score(y_train_encoded, y_pred_svm)
print("SVM Classifier")
print("Training Accuracy:", accuracy_svm)

SVM Classifier
Training Accuracy: 1.0


In [11]:
from sklearn.decomposition import PCA
pca = PCA(n_components=0.95)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)
model=SVC()
model.fit(X_train_pca, y_train_encoded)
y_pred = model.predict(X_test_pca)
accuracy = accuracy_score(y_test_encoded, y_pred)
print("SVM Classifier")
print("Accuracy:", accuracy)
print("Classification Report:")
print(classification_report(y_test_encoded, y_pred, target_names=label_encoder.classes_))

SVM Classifier
Accuracy: 0.7790697674418605
Classification Report:
                   precision    recall  f1-score   support

     Ariel_Sharon       1.00      0.29      0.44        14
     Colin_Powell       0.68      0.97      0.80        40
  Donald_Rumsfeld       0.93      0.74      0.82        34
    George_W_Bush       0.76      0.99      0.86       109
Gerhard_Schroeder       1.00      0.17      0.30        23
      Hugo_Chavez       0.00      0.00      0.00        11
       Tony_Blair       0.88      0.78      0.82        27

         accuracy                           0.78       258
        macro avg       0.75      0.56      0.58       258
     weighted avg       0.78      0.78      0.73       258



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [12]:
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV
parameters = [
        {"kernel":
            ["linear"],
            "C": [ 0.1, 1, 10, 100]},
        {"kernel":
            ["poly"],
            "degree": [2, 3, 4],
            "C": [ 0.1, 1, 10, 100]},
        {"kernel":
            ["rbf"],
            "gamma": ["auto", "scale"],
            "C": [ 0.1, 1, 10, 100]}
]

svm_clf = SVC()
print("tuning hyperparameters via grid search")
grid = GridSearchCV(estimator=SVC(), param_grid=parameters, n_jobs=-1)
grid.fit(X_train_pca,y_train_encoded)
print(f"grid search best score: {grid.best_score_ * 100:.2f}%")
print(f"grid search best parameters: {grid.best_params_}")



tuning hyperparameters via grid search


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()


grid search best score: 89.51%
grid search best parameters: {'C': 0.1, 'kernel': 'linear'}


In [13]:
model = SVC(C= 10, gamma= 'auto', kernel= 'rbf')
model.fit(X_train_pca,y_train_encoded)
y_pred_svm = model.predict(X_test_pca)
# print(classification_report(y_test_encoded, y_pred_svm))
print("SVM Classifier")
print("Classification Report:")
print(classification_report(y_test_encoded, y_pred_svm, target_names=label_encoder.classes_))

SVM Classifier
Classification Report:
                   precision    recall  f1-score   support

     Ariel_Sharon       1.00      0.50      0.67        14
     Colin_Powell       0.72      0.97      0.83        40
  Donald_Rumsfeld       0.91      0.85      0.88        34
    George_W_Bush       0.84      0.98      0.90       109
Gerhard_Schroeder       1.00      0.52      0.69        23
      Hugo_Chavez       1.00      0.09      0.17        11
       Tony_Blair       0.92      0.81      0.86        27

         accuracy                           0.84       258
        macro avg       0.91      0.68      0.71       258
     weighted avg       0.87      0.84      0.82       258



In [14]:
y_pred_svm = model.predict(X_train_pca)
accuracy_svm = accuracy_score(y_train_encoded, y_pred_svm)
print("SVM Classifier")
print("Accuracy:", accuracy_svm)

SVM Classifier
Accuracy: 1.0


In [15]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
lda = LDA(n_components=None)
X_train_lda = lda.fit_transform(X_train, y_train_encoded)
X_test_lda = lda.transform(X_test)

model.fit(X_train_lda, y_train_encoded)
y_pred_svm = model.predict(X_test_lda)
accuracy_svm = accuracy_score(y_test_encoded, y_pred_svm)
print("SVM Classifier")
print("Accuracy:", accuracy_svm)
print("Classification Report:")
print(classification_report(y_test_encoded, y_pred_svm, target_names=label_encoder.classes_))

SVM Classifier
Accuracy: 0.8333333333333334
Classification Report:
                   precision    recall  f1-score   support

     Ariel_Sharon       0.91      0.71      0.80        14
     Colin_Powell       0.95      0.88      0.91        40
  Donald_Rumsfeld       0.96      0.76      0.85        34
    George_W_Bush       0.75      0.98      0.85       109
Gerhard_Schroeder       0.92      0.52      0.67        23
      Hugo_Chavez       0.80      0.36      0.50        11
       Tony_Blair       0.95      0.78      0.86        27

         accuracy                           0.83       258
        macro avg       0.89      0.71      0.78       258
     weighted avg       0.86      0.83      0.83       258



In [16]:
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV
parameters = [
        {"kernel":
            ["linear"],
            "C": [ 0.1, 1, 10, 100]},
        {"kernel":
            ["poly"],
            "degree": [2, 3, 4],
            "C": [ 0.1, 1, 10, 100]},
        {"kernel":
            ["rbf"],
            "gamma": ["auto", "scale"],
            "C": [ 0.1, 1, 10, 100]}
]

svm_clf = SVC()
print("tuning hyperparameters via grid search")
grid = GridSearchCV(estimator=SVC(), param_grid=parameters, n_jobs=-1)
grid.fit(X_train_lda,y_train_encoded)
print(f"grid search best score: {grid.best_score_ * 100:.2f}%")
print(f"grid search best parameters: {grid.best_params_}")



tuning hyperparameters via grid search
grid search best score: 83.30%
grid search best parameters: {'C': 0.1, 'kernel': 'linear'}


In [17]:
model = SVC(C= 0.1, gamma= 'scale', kernel= 'rbf')
model.fit(X_train_lda,y_train_encoded)
y_pred_svm = model.predict(X_test_lda)
# print(classification_report(y_test_encoded, y_pred_svm))
print("SVM Classifier")
print("Classification Report:")
print(classification_report(y_test_encoded, y_pred_svm, target_names=label_encoder.classes_))

SVM Classifier
Classification Report:
                   precision    recall  f1-score   support

     Ariel_Sharon       1.00      0.50      0.67        14
     Colin_Powell       0.95      0.90      0.92        40
  Donald_Rumsfeld       1.00      0.65      0.79        34
    George_W_Bush       0.69      1.00      0.81       109
Gerhard_Schroeder       1.00      0.52      0.69        23
      Hugo_Chavez       1.00      0.09      0.17        11
       Tony_Blair       1.00      0.70      0.83        27

         accuracy                           0.80       258
        macro avg       0.95      0.62      0.70       258
     weighted avg       0.86      0.80      0.78       258



In [19]:
y_pred_svm = model.predict(X_train_lda)
accuracy_svm = accuracy_score(y_train_encoded, y_pred_svm)
print("SVM Classifier")
print("Accuracy:", accuracy_svm)

SVM Classifier
Accuracy: 0.8359223300970874
